### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-11-11 10:55:08.947056: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


#### Congifuring model

In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(150,150,3)))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)


2022-11-11 10:55:10.670377: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 10:55:10.677660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 10:55:10.678320: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 10:55:10.679307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

#### Q2

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

#### Data Generation and Model Fitting

In [4]:
train_generator = ImageDataGenerator(rescale=1./255)
val_generator = ImageDataGenerator(rescale=1./255)

In [5]:
train_data = train_generator.flow_from_directory(
    './train',
    batch_size=20,
    shuffle=True,
    class_mode='binary',
    target_size=(150, 150)
)

val_data = val_generator.flow_from_directory(
    './test',
    batch_size=20,
    shuffle=True,
    class_mode='binary',
    target_size=(150, 150)
)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [6]:
history = model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10


2022-11-11 10:55:13.097681: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-11 10:55:13.829856: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-11 10:55:13.830747: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-11 10:55:13.830784: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-11 10:55:13.831666: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-11 10:55:13.831754: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


80/80 [==============================] - 11s 107ms/step - loss: 0.6652 - accuracy: 0.5941 - val_loss: 0.6127 - val_accuracy: 0.6650
Epoch 2/10
80/80 [==============================] - 8s 105ms/step - loss: 0.5466 - accuracy: 0.7415 - val_loss: 0.4783 - val_accuracy: 0.8173
Epoch 3/10
80/80 [==============================] - 8s 105ms/step - loss: 0.4496 - accuracy: 0.8061 - val_loss: 0.4260 - val_accuracy: 0.7893
Epoch 4/10
80/80 [==============================] - 8s 104ms/step - loss: 0.4022 - accuracy: 0.8312 - val_loss: 0.3746 - val_accuracy: 0.8452
Epoch 5/10
80/80 [==============================] - 8s 105ms/step - loss: 0.3453 - accuracy: 0.8607 - val_loss: 0.3859 - val_accuracy: 0.8274
Epoch 6/10
80/80 [==============================] - 8s 105ms/step - loss: 0.3267 - accuracy: 0.8639 - val_loss: 0.3381 - val_accuracy: 0.8579
Epoch 7/10
80/80 [==============================] - 8s 106ms/step - loss: 0.2937 - accuracy: 0.8883 - val_loss: 0.3407 - val_accuracy: 0.8452
Epoch 8/10
80/80

#### Q3

In [7]:
round(np.median(history.history['accuracy']), 2)

0.86

#### Q4

In [8]:
round(np.std(history.history['loss']), 2)

0.14

#### Augmented Data

In [9]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest' 
)

train_data_augmented = train_generator.flow_from_directory(
    './train',
    batch_size=20,
    shuffle=True,
    class_mode='binary',
    target_size=(150, 150)
)

Found 1594 images belonging to 2 classes.


In [10]:
history_augmented = model.fit(train_data_augmented, epochs=10, validation_data=val_data)

Epoch 1/10
80/80 [==============================] - 15s 189ms/step - loss: 0.4479 - accuracy: 0.7974 - val_loss: 0.2966 - val_accuracy: 0.8883
Epoch 2/10
80/80 [==============================] - 15s 188ms/step - loss: 0.4266 - accuracy: 0.8093 - val_loss: 0.6477 - val_accuracy: 0.6954
Epoch 3/10
80/80 [==============================] - 15s 188ms/step - loss: 0.4239 - accuracy: 0.8068 - val_loss: 0.6025 - val_accuracy: 0.7360
Epoch 4/10
80/80 [==============================] - 15s 186ms/step - loss: 0.3871 - accuracy: 0.8199 - val_loss: 0.3947 - val_accuracy: 0.8071
Epoch 5/10
80/80 [==============================] - 15s 186ms/step - loss: 0.4038 - accuracy: 0.8174 - val_loss: 0.3269 - val_accuracy: 0.8655
Epoch 6/10
80/80 [==============================] - 15s 187ms/step - loss: 0.3781 - accuracy: 0.8338 - val_loss: 0.3634 - val_accuracy: 0.8325
Epoch 7/10
80/80 [==============================] - 15s 186ms/step - loss: 0.3652 - accuracy: 0.8488 - val_loss: 0.4091 - val_accuracy: 0.8198

#### Q5

In [11]:
round(np.mean(history_augmented.history['val_loss']), 2)

0.45

#### Q6

In [12]:
round(np.average(history_augmented.history['val_accuracy'][5:]), 2)

0.81